In [13]:
"""
PRE-REQUISITES - for demo, to be improved

- ANACONDA and PYTHON INSTALLED
- RENAME RP FILES to RP10, RP100, RP1000
- HAVE ADM data with proper ADM_CODE and ADM_NAME COLS FOR ALL LEVELS!
- SET THE STRUCTURE OF FOLDERS AS REQUESTED

"""


'\nPRE-REQUISITES - for demo, to be improved\n\n- ANACONDA and PYTHON INSTALLED\n- RENAME RP FILES to RP10, RP100, RP1000\n- HAVE ADM data with proper ADM_CODE and ADM_NAME COLS FOR ALL LEVELS!\n- SET THE STRUCTURE OF FOLDERS AS REQUESTED\n\n'

In [14]:
# Required libraries
import tempfile, os

import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm

import warnings

import rasterio
import xarray as xr
import rioxarray as rxr
from rasterstats import gen_zonal_stats

import requests
import json

import contextily as ctx
# from contextily import Place

# Addresses SSL error when interacting with worldpop data
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
# Common directories
DATA_DIR = "X:/Work/GeoTemp/CCDR/Nepal/data_script/"
ADM_bound = "X:/Work/GeoTemp/CCDR/Nepal/data_script/adm"


# Common directories
DATA_DIR = "../data/"
SAR_loc = "C:/development/CDCC-data/SAR/"

# Make cache dir if needed
CACHE_DIR = f"{DATA_DIR}cache/"
os.makedirs(CACHE_DIR, exist_ok=True)

In [16]:
def damage_factor(x):
    """A polynomial fit to average damage across multiple sectors relative 
    to water depth in meters in Asia.

    The sectors are commercial, industry, transport, agriculture, infrastructure and residential.

    Values are capped between 0 and 1, where values >= 6m = 1

    References
    ----------
    .. [1] JRC, 2017
    """
    return np.maximum(0.0, np.minimum(1.0, 0.00723*x**3 - 0.1*x**2 + 0.506*x))


In [17]:
# Stub for user input
# TODO: Make this nicer and more accessible for users

country = "NPL"
exp_cat = ["population", "land cover"]
# Population maps from worldpop are available for a range of years. Specify the year after checking for availability.
# population_year = "2020"

# Settings
agg_criteria = "mean"  # ["max", "mean"]
min_haz_threshold = 0.5  # determine min/max values from user-selected edges

In [18]:
# Testing data file locations
# TODO: Temp data store, to be replaced with a config spec (.env file?) before deployment

# pop_fn = f"{DATA_DIR}/cache/{fid}_{cache_fn}"
pop_fn = f"{DATA_DIR}/cache/WorldPop20_{country}_ppp_UNadj_constrained.tif"

# Flood data location (TODO: replace with pointer to downloaded data store)
flood_RP_data_loc = f"X:/Work/GeoTemp/CCDR/Nepal/data_script/"
flood_RP_data_loc = f"C:/development/CDCC-data/SAR/HZD/Flood/{country}/"

In [19]:
country_code_map = {
    "NPL": 175  # TODO: Add others
}

In [20]:
# Load or save ISO3 country list
iso3_path = f"{DATA_DIR}cache/iso3.json"
if not os.path.exists(iso3_path):
    resp = json.loads(requests.get(f"https://www.worldpop.org/rest/data/pop/wpgp?iso3={country}").text)

    with open(iso3_path, 'w') as outfile:
        json.dump(resp, outfile)
else:
    with open(iso3_path, 'r') as infile:
        resp = json.load(infile)


# TODO: User to select population data set
# Target population data files are extracted from the JSON list downloaded above
metadata = resp['data'][1]
data_src = metadata['files']

In [21]:
# Save population data to cache location
for data_fn in tqdm(data_src):
    fid = metadata['id']
    cache_fn = os.path.basename(data_fn)

    # Look for indicated file in cache directory
    # Use the data file if it is found, but warn the user. 
    # (if data is incorrect or corrupted, they should delete it from cache)
    if f"{fid}_{cache_fn}" in os.listdir(f"{DATA_DIR}/cache"):
        warnings.warn(f"Found {fid}_{cache_fn} in cache, skipping...")
        continue

    # Write to cache file if not found
    with open(f"{DATA_DIR}/cache/{fid}_{cache_fn}", "wb") as handle:
        response = requests.get(data_fn)
        handle.write(response.content)

  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\takuy\AppData\Local\Temp\ipykernel_18136\695879328.py:10: UserWarning: Found 1591_npl_ppp_2001.tif in cache, skipping...
  warnings.warn(f"Found {fid}_{cache_fn} in cache, skipping...")
100%|██████████| 1/1 [00:00<00:00, 142.88it/s]


Load country boundaries from ADM geopackage file which includes ISO3 code related to country name

In [22]:
# country_bounds = gpd.read_file(os.path.join(ADM_bound, "NPL_ADM.gpkg"))
country_bounds = gpd.read_file(os.path.join(DATA_DIR, "NPL_ADM.gpkg"))

In [30]:
valid_RPs = [10, 100, 1000]

# Open population dataset
pop_data = rxr.open_rasterio(pop_fn)

# Indicate -1 values as representing no data.
pop_data.rio.write_nodata(-1, inplace=True)

# Load ADM2 based on country code value
adm_dataset = gpd.read_file(os.path.join(DATA_DIR, "NPL_ADM.gpkg"), layer="ADM3")
adm_data = adm_dataset.loc[adm_dataset.ADM0_CODE == country_code_map[country], :]

# Prep result structure
pop_sum_cols = [f"RP{rp_i}_pop_sum" for rp_i in valid_RPs]
EAI_cols = [f"RP{rp_i}_EAI" for rp_i in valid_RPs]
result_df = adm_data.loc[:, ["ADM3_CODE", "ADM3_NAME", "geometry"]]
result_df.loc[:, pop_sum_cols + EAI_cols] = 0

pop_bounds = pop_data.rio.bounds()

crs = result_df.crs
for rp in valid_RPs:
    
    # Get total population for each ADM2 region
    pop_per_ADM = gen_zonal_stats(vectors=adm_data["geometry"], raster=pop_fn, stats=["sum"])
    
    result_df["ADM3_Pop"] = [x['sum'] for x in pop_per_ADM]

    # Load corresponding flood dataset
    flood_data = rxr.open_rasterio(flood_RP_data_loc+f"RP{rp}.tif")

    # Reproject and clip raster to same bounds as population data
    flood_data = flood_data.rio.reproject_match(pop_data)

    # Get raw array values for population and flood
    fld_array = flood_data[0].values
    fld_array[fld_array < min_haz_threshold] = np.nan  # Set values below min threshold to nan
    # fld_array[fld_array > max_haz_threshold] = max_haz_threshold  # Cap large values to maximum threshold value

    # Assign impact factor
    impact_array = damage_factor(fld_array)

    # Create raster from array
    impact_rst = xr.DataArray(np.array([impact_array]).astype(np.float32), coords=flood_data.coords, dims=flood_data.dims)
    # impact_rst.plot()  # to preview

    # Impact x Pop
    # Calculate affected population in ADM3
    affected_pop = pop_data.where(impact_rst.values > 0)
    affected_pop = affected_pop.where(affected_pop > 0)

    # Probability of return period
    # Essentially the same as 1/RP, but accounts for cases where RP == 1
    RPp = 1 - np.exp(-1/rp)

    RPi_EAI = affected_pop * RPp

    # Get affected population per ADM
    affected_pop_per_ADM = gen_zonal_stats(vectors=adm_data["geometry"], raster=affected_pop.data[0], 
                                           stats=["sum"], affine=affected_pop.rio.transform(), nodata=np.nan)
    result_df[f"RP{rp}_pop_sum"] = [x['sum'] for x in affected_pop_per_ADM]


    EAI_per_ADM = gen_zonal_stats(vectors=adm_data["geometry"], raster=RPi_EAI.data[0], 
                                  stats=["sum"], affine=RPi_EAI.rio.transform(), nodata=np.nan)
    result_df[f"RP{rp}_EAI"] = [x['sum'] for x in EAI_per_ADM]

        
# Sum all EAI to get total EAI across all RPs
result_df.loc[:, "Pop_EAI"] = result_df.loc[:, result_df.columns.str.contains('_EAI')].sum(axis=1)

# Calculate Pop_EAI%
result_df.loc[:, "Pop_EAI%"] = result_df.loc[:, "Pop_EAI"] / result_df.loc[:, "ADM3_Pop"]  # Percent affected population per year

# Aggregated to ADM1
agg_func = getattr(np, agg_criteria)
result_df.loc[:, f"ADM1_agg_{agg_criteria}"] = agg_func(result_df.loc[:, "Pop_EAI%"])

# Write table of total population in each class, in each ADM2
result_df.to_csv(f"{country}_functional_example_results.csv", index=False)

# Export geopackage
result_df.to_file(f"{country}_ADM3_test2.gpkg")
